In [45]:
import subprocess
import time

from pathlib import Path

import pandas as pd

In [46]:
def build_dtree():
    subprocess.run(["cargo", "build", "--release"])
    
build_dtree()

    Finished `release` profile [optimized] target(s) in 0.06s


In [47]:
def run_command(command: list[str]) -> tuple[float, int, str]:
    """
    Runs a command and captures execution time, peak memory usage, and output.
    
    Args:
        command (list[str]): Command to execute as a list of arguments.

    Returns:
        tuple[float, int, str]: Execution time (seconds), peak memory usage (KB), and output.
    """
    start_time = time.time()
    process = subprocess.Popen(
        command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
    )
    stdout, stderr = process.communicate()
    execution_time = time.time() - start_time

    # Parse memory usage from /usr/bin/time format
    memory_usage = 0
    if stderr:
        for line in stderr.splitlines():
            if "maxresident" in line:
                memory_usage = int(line.split()[-1])

    return execution_time, memory_usage, stdout

In [48]:
def benchmark_programs(test_dirs: list[str], my_program: str, output_file: str) -> None:
    """
    Benchmarks the user's program and the `tree` command across multiple directories
    with various argument combinations.

    Args:
        test_dirs (list[str]): List of directories to benchmark on.
        my_program (str): Path to the user's tree-like program.
        output_file (str): File path to save the benchmark results.

    Returns:
        None
    """
    results = []

    # Define argument combinations to test
    my_program_args = [
        [],  # Default arguments
        ["-L", "2"], 
        # ["-L", "5"],  # Limit depth
        ["-a"],  # Show hidden files
        # ["-f"],  # Full paths
        ["-L", "3", "-a", "-f"],  # Combination of all flags
    ]

    tree_program_args = [
        [],  # Default arguments
        ["-L", "2"],  # Limit depth
        # ["-L", "5"],  # Limit depth
        ["-a"],  # Show hidden files
        # ["-f"],  # Full paths
        ["-L", "3", "-a", "-f"],  # Combination of all flags
    ]

    for directory in test_dirs:
        print(f"Benchmarking on: {directory}")

        for my_args, tree_args in zip(my_program_args, tree_program_args):
            # Run my_program with arguments
            my_command = [my_program] + my_args + [directory]
            my_time, my_memory, my_output = run_command(my_command)

            # Run tree command with arguments
            tree_command = ["tree"] + tree_args + [directory]
            tree_time, tree_memory, tree_output = run_command(tree_command)

            # Validate outputs
            output_match = "Match" if my_output == tree_output else "Mismatch"

            # Save results
            results.append({
                "Directory": directory,
                "Program": "dtree",
                "Arguments": " ".join(my_args),
                "Time(s)": my_time,
                "Memory(KB)": my_memory,
                "OutputMatch": output_match,
            })

            results.append({
                "Directory": directory,
                "Program": "tree",
                "Arguments": " ".join(tree_args),
                "Time(s)": tree_time,
                "Memory(KB)": tree_memory,
                "OutputMatch": "-",
            })

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    print(f"Benchmark results saved to {output_file}")


In [49]:
# List of directories to benchmark on
test_directories = [
    str(Path.cwd().parent),
    str(Path().home())
    ]

# Path to the user's tree-like program
user_program_path = str(Path().cwd().parent / "target/release/dtree")

# Output CSV file
output_csv = str(Path.cwd() / "results.csv")

# Run the benchmark
benchmark_programs(test_directories, user_program_path, output_csv)

Benchmarking on: /Users/roman/Repos/dtree
Benchmarking on: /Users/roman
Benchmark results saved to /Users/roman/Repos/dtree/benchmarks/results.csv
